Importing packages

In [1]:
from dolfinx import fem, mesh
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from configs import *
import utils
import ufl
from mpi4py import MPI
import numpy as np

Creating a mesh, defining function spaces, functions and test functions

In [2]:
# Create a mesh, elements and spaces
domain = utils.create_mesh(N=10)
vector_element = ufl.VectorElement("Lagrange", domain.ufl_cell(), 2, 2)
element = ufl.FiniteElement("Lagrange", domain.ufl_cell(), 1)

W = fem.FunctionSpace(domain, vector_element)
V = fem.FunctionSpace(domain, element)

# Define test functions
phi, psi = ufl.TestFunctions(W)
theta = ufl.TestFunction(V)

# Define functions
v, v_n, v_nn = fem.Function(W), fem.Function(W), fem.Function(W)
w, w_n, w_nn = fem.Function(V), fem.Function(V), fem.Function(V)

V_m, u = v.split()
V_m_n, u_n = v_n.split()
V_m_nn, u_nn = v_nn.split()

v_tilde = fem.Function(W)
V_m_tilde, u_tilde = v_tilde.split()

x = ufl.SpatialCoordinate(domain)
d = domain.topology.dim
t = 0

# Define meshtags
num_cells = domain.topology.index_map(domain.topology.dim).size_local
body_cells = np.arange(num_cells)
markers = np.zeros(num_cells)

heart_cells = mesh.locate_entities(domain, d, utils.heart_marker)
markers[heart_cells] = np.full_like(heart_cells, 1)
tags = mesh.meshtags(domain, d, body_cells, markers)

# Define new measure such that dx(0) integrates torso and dx(1) integrates heart
dx = ufl.Measure("dx", domain, subdomain_data=tags)

Defining model parameters

In [3]:
# Gating variable
def g(V_m: fem.Function, w: fem.Function):
    condition = ufl.lt(V_m, V_GATE)
    true_statement = w / TAU_OPEN - 1 / TAU_OPEN / (V_MAX - V_MIN) ** 2
    false_statement = w / TAU_CLOSE
    return ufl.conditional(condition, true_statement, false_statement)


# Fibre orientations
fibres = ufl.as_vector(
    [
        x[1] / ufl.sqrt(x[0] ** 2 + x[1] ** 2 + 1),
        -x[0] / ufl.sqrt(x[0] ** 2 + x[1] ** 2 + 1),
    ],
)


# Ionic current
def I_ion(V_m: fem.Function, w: fem.Function):
    return -w / TAU_IN * (V_m - V_MIN) ** 2 * (V_MAX - V_m) / (
        V_MAX - V_MIN
    ) + 1 / TAU_OUT * (V_m - V_MIN) / (V_MAX - V_MIN)


# Applied stimulus - custom? - may need change
def I_app(t, size=1, t_act=0.1):
    return ufl.exp(
        -(((x[0] - Hx) / size) ** 2) - ((x[1] - Hy) / size) ** 2 - (t / t_act) ** 2
    )


# Conductivities
sigma_i = SIGMA_IT * ufl.Identity(d) + (SIGMA_IL - SIGMA_IT) * ufl.outer(fibres, fibres)
sigma_e = SIGMA_ET * ufl.Identity(d) + (SIGMA_EL - SIGMA_ET) * ufl.outer(fibres, fibres)
sigma_t = SIGMA_TLT * ufl.Identity(d)

Setting up initial conditions

In [4]:
# Initial conditions
V_m_n.vector.set(V_MIN)
w_n.vector.set(1 / (V_MAX - V_MIN) ** 2)
V_m_nn.vector.set(V_MIN)
w_nn.vector.set(1 / (V_MAX - V_MIN) ** 2)

Implementing FEM steps for a solution

In [5]:
# Step 1
def step_1(V_m_tilde):
    V_m_tilde.x.array[:] = 2 * V_m_n.x.array[:] - V_m_nn.x.array[:]


# Step 2
def step_2():
    F_w = (
        (1 / dt * (3 / 2 * w - 2 * w_n + 1 / 2 * w_nn) + g(V_m_tilde, w))
        * theta
        * dx(1)
    )

    problem = NonlinearProblem(F_w, w)
    solver = NewtonSolver(MPI.COMM_WORLD, problem)
    solver.rtol = 1e-0
    solver.atol = 1e-0
    solver.max_it = 5
    log.set_log_level(log.LogLevel.INFO)
    solver.solve(w)


# Step 3
def step_3():
    F_v = (
        A_m * C_m / dt * (3 / 2 * V_m - 2 * V_m_n + 1 / 2 * V_m_nn) * phi * dx(1)
        + ufl.inner(ufl.dot(sigma_i, ufl.grad(V_m + u)), ufl.grad(phi)) * dx(1)
        - A_m * (I_app(t) - I_ion(V_m_tilde, w)) * phi * dx(1)
    )

    F_v += (
        ufl.inner(ufl.dot(sigma_i + sigma_e, ufl.grad(u)), ufl.grad(psi)) * dx(1)
        + ufl.inner(ufl.dot(sigma_i, ufl.grad(V_m)), ufl.grad(psi)) * dx(1)
        + ufl.inner(ufl.dot(sigma_t, ufl.grad(u)), ufl.grad(psi)) * dx(0)
    )

    problem = NonlinearProblem(F_v, v)
    solver = NewtonSolver(MPI.COMM_WORLD, problem)
    solver.rtol = 1e-0
    solver.atol = 1e-0
    solver.max_it = 5
    log.set_log_level(log.LogLevel.INFO)
    solver.solve(v)


def time_step(t):
    t += dt
    step_1(V_m_tilde)
    step_2()
    step_3()


time_step(t)

TypeError: compute_integration_domains(): incompatible function arguments. The following argument types are supported:
    1. (integral_type: dolfinx::fem::IntegralType, meshtags: dolfinx.cpp.mesh.MeshTags_int32) -> List[Tuple[int, List[int]]]

Invoked with: <IntegralType.cell: 0>, <dolfinx.cpp.mesh.MeshTags_float64 object at 0x7fea0f9fff70>